# Reading data from port API

URL = http://data.portic.fr/api/ports?param=&shortenfields=false&both_to=false&date=1787


In [ ]:
import json
import pandas as pd
import geopandas as gpd

import requests

In [ ]:
data_local = False
if data_local : 
    #filename = "C:\Travail\Enseignement\Cours_M2_python\Exemple\ports.json"
    filename = "C:/Travail/Enseignement/Cours_M2_python/2023/code/resultats/export3_port_geojson.geojson"
    output = open(filename, "r")
    data = json.load(output)
else:
    
    ## https://requests.readthedocs.io/en/latest/
    url = "http://data.portic.fr/api/ports/?shortenfields=false&both_to=false&date=1787"
    r = requests.get(url)
    #print(r.text)
    print(type(r.json()))
    data = r.json()

In [ ]:
data

In [ ]:
#data is of <class 'list'>
df = pd.DataFrame(data)

print(df.shape) #(1700, 25)
#print(df.head())

In [ ]:
print(df.admiralty.unique())
print(df.admiralty.unique().size)
print(df.admiralty.isnull().values.any()) #True 



In [ ]:
print(df.state_1789_fr.isnull().values.any()) #True 


## Dealing with null values

In [ ]:
# Dealing with null values
df.admiralty.isnull().values.any() #True 
values = {'admiralty': 'X', 'state_1789_fr' : 'UNKNOWN'}
#values = {'admiralty': 'X'}
df = df.fillna(value=values)
df.admiralty.isnull().values.any() #False 

In [ ]:
# Listing of admiralties
print(df.admiralty.unique().size) #71
df.admiralty.unique() 


## Group by - agrégat

In [ ]:
# How many ports by admiralty ?
res = df.groupby('admiralty')['ogc_fid'].count()
print(res)


type(res)
res.to_csv('nb_port_par_amiraute.csv')

In [ ]:
res.to_excel('nb_port_par_amiraute.xls')

In [ ]:
# How many ports by state_1789_fr ?
res = df.groupby('state_1789_fr')['ogc_fid'].count()
print(res)
#Attention, il reste des valeurs null pour les états
print(df.state_1789_fr.isnull().values.any()) #True 

# Map with folium

In [ ]:
import folium

### Solution 01

In [ ]:
m = folium.Map(location=[46.34015, 2.60254], zoom_start=6, tiles="cartodb positron")
#filter
subset=df[df['state_1789_fr']=='France']
subset.reset_index(drop=True, inplace=True)

a=subset.toponym
b=subset.long
c=subset.lat

for i in range(len((subset.toponym))):
    folium.Marker(
        location=[c[i], b[i]],
        popup=a[i],
        icon=folium.Icon(icon="Waypoint"),
    ).add_to(m)



m.save("map_portic_01.html")

### Solution 02

In [ ]:
import numpy as np

longitudes = np.array(df.loc[df['state_1789_fr'] == 'France']['long'])
latitudes = np.array(df.loc[df['state_1789_fr'] == 'France']['lat'])
names = np.array(df.loc[df['state_1789_fr'] == 'France']['toponym'])

# Code carte

m = folium.Map(location=[49.49437, 0.107929], zoom_start=3, tiles="cartodb positron") #, tiles="Stamen Terrain"


for i in range(len(latitudes)):
    folium.CircleMarker(
    location=[latitudes[i], longitudes[i]],
    radius=5,
    popup=names[i],
    color="#3186cc",
    fill=True,
    fill_color="#3186cc",
).add_to(m)

m

### Solution 03

In [ ]:
m = folium.Map(location=[46.156089, -1.156176], tiles="cartodb positron" ) #tiles="Stamen Watercolor"
m.add_child(folium.LatLngPopup())
m.add_child(folium.ClickForMarker(popup="Waypoint"))

for x,y,name,state,adm in zip(df.long,df.lat,df.toponyme_standard_fr,df.state_1789_fr,df.admiralty):
    if state=='France':
        folium.Marker([y, x], popup=f"<i>{adm}</i>", tooltip=name,icon=folium.Icon(color="green")).add_to(m)
    else:
        folium.Marker([y, x], popup=f"<i>{adm}</i>", tooltip=name).add_to(m)
m

## A la manière de mes élèves

- Avec un fond standard cartodb positron

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Centrer sur 49.49437, 0.107929

m = folium.Map(location=(49.49437, 0.107929), zoom_start=4, tiles="cartodb positron", width=1200, height=800)



## add markers for chef-lieux 
group_1 = folium.FeatureGroup("ports").add_to(m)


#popup_content = '<table><tr><td>Nom</td><td>{0}</td></tr><tr><td>Statut</td><td>{1}</td></tr><tr><td>Amirauté</td><td>{2}</td></tr><tr><td>Pays 1789</td><td>{3}</td></tr></table>'

for index, row in df.iterrows() :
    #position des markers  : [latitude, longitude]
    folium.Marker(
        location=[row.y, row.x], #geoms[0].
        tooltip=row.toponyme_standard_fr,
        popup=row.admiralty,
        #popup_content.format(row.toponyme_standard_fr, row.status, row.admiralty, row.state_1789_fr),
        icon=folium.Icon(color="green"),
    ).add_to(group_1)
    

folium.LayerControl().add_to(m)

m

## Enrichir la carte

- Rajouter une fonction de clustering
- mettre un fond Stamen (tile)

https://python-visualization.github.io/folium/latest/user_guide/plugins/marker_cluster.html

In [ ]:
import folium
from folium.plugins import MarkerCluster
import xyzservices

smithsonian_provider = xyzservices.TileProvider (
    name="Stamen maps, hosted by Smithsonian",
    url="https://watercolormaps.collection.cooperhewitt.org/tile/watercolor/{z}/{x}/{y}.jpg",
    attribution="(C) Stamen Design",
)

# Centrer sur 49.49437, 0.107929

m = folium.Map(location=(49.49437, 0.107929), zoom_start=4, tiles=smithsonian_provider, width=1200, height=800)

marker_cluster = MarkerCluster().add_to(m)


## add markers for chef-lieux 
group_1 = folium.FeatureGroup("ports").add_to(m)


#popup_content = '<table><tr><td>Nom</td><td>{0}</td></tr><tr><td>Statut</td><td>{1}</td></tr><tr><td>Amirauté</td><td>{2}</td></tr><tr><td>Pays 1789</td><td>{3}</td></tr></table>'

for index, row in df.iterrows() :
    #position des markers  : [latitude, longitude]
    folium.Marker(
        location=[row.y, row.x], #geoms[0].
        tooltip=row.toponyme_standard_fr,
        popup=row.admiralty,
        #popup_content.format(row.toponyme_standard_fr, row.status, row.admiralty, row.state_1789_fr),
        icon=folium.Icon(color="green"),
    ).add_to(marker_cluster)
    

folium.LayerControl().add_to(m)

m

### Solution de Geoffrey
Usage d'icônes personnalisées

In [ ]:
dt_lr = df[df.admiralty == 'La Rochelle']
dt_fr = df[df.state_1789_fr == 'France']
dt_autre = df[df.state_1789_fr != 'France']

svg_boat_fr = '''
<svg xmlns="http://www.w3.org/2000/svg" height="24" width="27" viewBox="0 0 576 512"><!--!Font Awesome Free 6.6.0 by @fontawesome - https://fontawesome.com License - https://fontawesome.com/license/free Copyright 2024 Fonticons, Inc.--><path fill="#171593" d="M320 96a32 32 0 1 1 -64 0 32 32 0 1 1 64 0zm21.1 80C367 158.8 384 129.4 384 96c0-53-43-96-96-96s-96 43-96 96c0 33.4 17 62.8 42.9 80L224 176c-17.7 0-32 14.3-32 32s14.3 32 32 32l32 0 0 208-48 0c-53 0-96-43-96-96l0-6.1 7 7c9.4 9.4 24.6 9.4 33.9 0s9.4-24.6 0-33.9L97 263c-9.4-9.4-24.6-9.4-33.9 0L7 319c-9.4 9.4-9.4 24.6 0 33.9s24.6 9.4 33.9 0l7-7 0 6.1c0 88.4 71.6 160 160 160l80 0 80 0c88.4 0 160-71.6 160-160l0-6.1 7 7c9.4 9.4 24.6 9.4 33.9 0s9.4-24.6 0-33.9l-56-56c-9.4-9.4-24.6-9.4-33.9 0l-56 56c-9.4 9.4-9.4 24.6 0 33.9s24.6 9.4 33.9 0l7-7 0 6.1c0 53-43 96-96 96l-48 0 0-208 32 0c17.7 0 32-14.3 32-32s-14.3-32-32-32l-10.9 0z"/></svg>
'''

svg_boat_autre = '''
<svg xmlns="http://www.w3.org/2000/svg" height="24" width="27" viewBox="0 0 576 512"><!--!Font Awesome Free 6.6.0 by @fontawesome - https://fontawesome.com License - https://fontawesome.com/license/free Copyright 2024 Fonticons, Inc.--><path fill="#931515" d="M320 96a32 32 0 1 1 -64 0 32 32 0 1 1 64 0zm21.1 80C367 158.8 384 129.4 384 96c0-53-43-96-96-96s-96 43-96 96c0 33.4 17 62.8 42.9 80L224 176c-17.7 0-32 14.3-32 32s14.3 32 32 32l32 0 0 208-48 0c-53 0-96-43-96-96l0-6.1 7 7c9.4 9.4 24.6 9.4 33.9 0s9.4-24.6 0-33.9L97 263c-9.4-9.4-24.6-9.4-33.9 0L7 319c-9.4 9.4-9.4 24.6 0 33.9s24.6 9.4 33.9 0l7-7 0 6.1c0 88.4 71.6 160 160 160l80 0 80 0c88.4 0 160-71.6 160-160l0-6.1 7 7c9.4 9.4 24.6 9.4 33.9 0s9.4-24.6 0-33.9l-56-56c-9.4-9.4-24.6-9.4-33.9 0l-56 56c-9.4 9.4-9.4 24.6 0 33.9s24.6 9.4 33.9 0l7-7 0 6.1c0 53-43 96-96 96l-48 0 0-208 32 0c17.7 0 32-14.3 32-32s-14.3-32-32-32l-10.9 0z"/></svg>
'''


m = folium.Map()

for index, row in dt_fr.iterrows():  # Utilisation de iterrows pour parcourir chaque ligne du DataFrame
    folium.Marker(
        location=[row.y, row.x],  # Latitude et longitude
        tooltip=row.toponyme_standard_en,  # Tooltip affiché au survol
        popup=f"Location: {row.toponyme_standard_en}",  # Popup à afficher lorsque le marqueur est cliqué
        icon=folium.DivIcon(html=svg_boat_fr),  # Utilisation de DivIcon avec le SVG de bateau
    ).add_to(m)
    
for index, row in dt_autre.iterrows():  # Utilisation de iterrows pour parcourir chaque ligne du DataFrame
    folium.Marker(
        location=[row.y, row.x],  # Latitude et longitude
        tooltip=row.toponyme_standard_en,  # Tooltip affiché au survol
        popup=f"Location: {row.toponyme_standard_en}",  # Popup à afficher lorsque le marqueur est cliqué
        icon=folium.DivIcon(html=svg_boat_autre),  # Utilisation de DivIcon avec le SVG de bateau
    ).add_to(m)
 
m   
#m.save('map_icones_ancres.html')

## Sauver une carte dans un fichier image

In [ ]:
#Save the map to PNG 

#https://stackoverflow.com/questions/53565979/export-a-folium-map-as-a-png
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('carto_ports.png')

# Post cours - introduction aux geodataframes

In [ ]:
print(df.columns)
df.rename(columns={'x':'long', 'y':'lat'}, inplace=True)
print(df.columns)

In [ ]:
# Build a true geopandas dataframe
#https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html 
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.long, df.lat), crs="EPSG:4326"
)

print(gdf.columns)
type(gdf) #pandas.core.indexes.base.Index

In [ ]:
ports = gdf[['uhgs_id', 'lat', 'long', 'toponyme_standard_fr', 'admiralty', 'status', 'province', 'substate_1789_fr', 'state_1789_fr', 'geometry']]
print(ports.shape)#1700 rows, 10 cols
type(ports) #geopandas.geodataframe.GeoDataFrame

## Map with geopandas

In [ ]:
# Map with geopandas
import matplotlib.pyplot as plt

world = gpd.read_file("C:/Travail/Enseignement/Cours_M2_python/2023/data/ne_earth_physical_land/ne_110m_land.shp")


fig, ax = plt.subplots()
world.plot(ax=ax, color='white', edgecolor='black')
ports.plot(ax=ax, marker='o', color='red', markersize=5)
plt.show()


## Map with folium

In [ ]:
import folium

# Centrer sur 49.49437, 0.107929

m = folium.Map(location=(49.49437, 0.107929), zoom_start=4, tiles="cartodb positron", width=1200, height=800)


## add markers for chef-lieux 
group_1 = folium.FeatureGroup("ports").add_to(m)


popup_content = '<table><tr><td>Nom</td><td>{0}</td></tr><tr><td>Statut</td><td>{1}</td></tr><tr><td>Amirauté</td><td>{2}</td></tr><tr><td>Pays 1789</td><td>{3}</td></tr></table>'

for index, row in ports.iterrows() :
    #position des markers  : [latitude, longitude]
    folium.Marker(
        location=[row.geometry.coords[0][1], row.geometry.coords[0][0]], #geoms[0].
        tooltip=row.toponyme_standard_fr,
        popup=popup_content.format(row.toponyme_standard_fr, row.status, row.admiralty, row.state_1789_fr),
        icon=folium.Icon(color="green"),
    ).add_to(group_1)
    
group_2 = folium.FeatureGroup("Monde").add_to(m)


for _, r in world.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "#00000000", "color": "#00FFFFFF"})
    #folium.Popup(r["zone_etude"]).add_to(geo_j)
    geo_j.add_to(group_2)
    
folium.LayerControl().add_to(m)

m